In [1]:
import numpy as np
import joblib

import params as p
import functions as f

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

In [2]:
train, test, val = f.load_split_datasets(part='03')

In [3]:
X_train, y_train = f.split_data_X_y(train)
X_test, y_test = f.split_data_X_y(test)
X_val, y_val = f.split_data_X_y(val)

In [4]:
rf = joblib.load(p.OUTPUTS_PATH + 'model_rf.pkl')

In [5]:
f.evaluate_model(rf, X_test, y_test)

rmse : 215.91885368701548
mae : 136.12454437869823
r2 : 0.8859423611477213


In [6]:
rf_1 = RandomForestRegressor(random_state=p.RANDOM_STATE)

In [7]:
param_space = {
    "max_depth": np.arange(2, 15),
    "n_estimators": np.arange(100, 5000, 50),
    # "min_samples_leaf": np.arange(2, 16, 1),
    # "min_samples_split": np.arange(0.1, 1, 0.05),
    # "max_features": ["auto", "sqrt", "log2"],
}

# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=p.RANDOM_STATE)

rs = RandomizedSearchCV(
    estimator = rf,
    param_distributions = param_space,
    n_jobs = -1,
    scoring='neg_mean_squared_error',
    cv = 3,
    random_state = p.RANDOM_STATE)

rs.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=73),
                   n_jobs=-1,
                   param_distributions={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'n_estimators': array([ 100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100, 1150,
       1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700,
       1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250,
       2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800,
       2850, 2900, 2950, 3000, 3050, 3100, 3150, 3200, 3250, 3300, 3350,
       3400, 3450, 3500, 3550, 3600, 3650, 3700, 3750, 3800, 3850, 3900,
       3950, 4000, 4050, 4100, 4150, 4200, 4250, 4300, 4350, 4400, 4450,
       4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950])},
                   random_state=73, scoring='neg_mean_squared_error')

In [8]:
tuned_params = rs.best_params_
tuned_params['random_state'] = p.RANDOM_STATE

In [9]:
rf_tuned = RandomForestRegressor(**tuned_params)
rf_tuned.fit(X_train, y_train)

RandomForestRegressor(max_depth=11, n_estimators=1900, random_state=73)

In [10]:
f.evaluate_model(rf_tuned, X_test, y_test)

rmse : 220.93317345485215
mae : 140.59335835499837
r2 : 0.8805832887234577
